# Lightweight Fine-Tuning Project

Description of choices in this soluton.

**PEFT technique:** LoRA (Low-Rank Adaptation) implemented through the Hugging Face PEFT framework.

**Model:** The pre-trained DistilBERT model (distilbert-base-uncased) is adapted for sequence classification tasks using LoRA.

**Evaluation approach:** The Hugging Face Trainer class is used for evaluation.

**Fine-tuning dataset:** The fine-tuning is performed on the SMS Spam dataset from the Hugging Face datasets library.

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
# Install the required version of datasets in case you have an older version
# You will need to choose "Kernel > Restart Kernel" from the menu after executing this cell
! pip install -q "datasets==2.15.0"

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
# Load the sms_spam dataset
# See: https://huggingface.co/datasets/sms_spam

from datasets import load_dataset

# The sms_spam dataset only has a train split, so we use the train_test_split method to split it into train and test
dataset = load_dataset("sms_spam", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]

# View the dataset characteristics
dataset["train"]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

Dataset({
    features: ['sms', 'label'],
    num_rows: 4459
})

In [3]:
# Inspect the first example. Do you think this is spam or not?
dataset["train"][0]

{'sms': 'Had your mobile 10 mths? Update to the latest Camera/Video phones for FREE. KEEP UR SAME NUMBER, Get extra free mins/texts. Text YES for a call\n',
 'label': 1}

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["sms"], truncation=True), batched=True
    )

# Inspect the available columns in the dataset
tokenized_dataset["train"]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/4459 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4459
})

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [5]:
# Create a LoRA configuration

In [6]:
from peft import LoraConfig
# Define LoRA configuration
config = LoraConfig(
    r=8,  # Attention rank
    lora_alpha=32,  # Scaling factor
    target_modules=["c_fc", "c_proj"],  # Adjusted for DistilBERT
    lora_dropout=0.1,  # Dropout for regularization
    bias="none",  # No additional biases
    task_type="SEQ_CLS",  # Sequence classification task
)

In [7]:
# Convert the Base model to PEFT model

In [8]:
from transformers import AutoModelForSequenceClassification
from peft import get_peft_model

# Load the pre-trained DistilBERT model
base_model = AutoModelForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)

# Apply LoRA configuration
model = get_peft_model(base_model, config)

# confirm that model is using parameter-efficient fine-turning
model.print_trainable_parameters()

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 887,808 || all params: 125,327,616 || trainable%: 0.7083897614393303


In [9]:
# GPT-2 model uses the pad token
model.config.pad_token_id = tokenizer.pad_token_id

In [10]:
print(model)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GPT2ForSequenceClassification(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Conv1D()
              (c_proj): Linear(
                in_features=768, out_features=768, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=768, bias=False)
                )
                (lora_embedding_A): ParameterDict()
 

In [11]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

In [12]:
# Train the model
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam",
        learning_rate=5e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Run the training
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.205700,0.099384,0.985650


Checkpoint destination directory ./data/spam_not_spam/checkpoint-558 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=558, training_loss=0.19219607947975076, metrics={'train_runtime': 47.9781, 'train_samples_per_second': 92.938, 'train_steps_per_second': 11.63, 'total_flos': 119442985482240.0, 'train_loss': 0.19219607947975076, 'epoch': 1.0})

In [13]:
# Evaluate the model

In [14]:
results = trainer.evaluate()
print(f"Evaluation results: {results}")

Evaluation results: {'eval_loss': 0.09938406199216843, 'eval_accuracy': 0.9856502242152466, 'eval_runtime': 4.7992, 'eval_samples_per_second': 232.328, 'eval_steps_per_second': 14.586, 'epoch': 1.0}


###  ⚠️ IMPORTANT ⚠️

Due to workspace storage constraints, you should not store the model weights in the same directory but rather use `/tmp` to avoid workspace crashes which are irrecoverable.
Ensure you save it in /tmp always.

In [15]:
# Saving the model
model.save_pretrained("/tmp/lora_gpt2")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [16]:
# Reload saved LoRA weights
from peft import AutoPeftModelForSequenceClassification

In [17]:
# Load the PEFT model
inference_model = AutoPeftModelForSequenceClassification.from_pretrained(
    "/tmp/lora_gpt2"
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
# Perform inference
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
inputs = tokenizer("Is this message spam?", return_tensors="pt")
outputs = inference_model(**inputs)
print(outputs)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

SequenceClassifierOutputWithPast(loss=None, logits=tensor([[2.1094, 4.9135]], grad_fn=<IndexBackward0>), past_key_values=((tensor([[[[-1.5757,  2.3004,  0.6426,  ..., -0.3471, -0.6349,  2.2196],
          [-2.7414,  2.9494,  2.2881,  ..., -0.3083, -1.3036,  1.4183],
          [-2.7456,  2.4031,  1.2684,  ...,  0.0745,  0.0184,  1.5793],
          ...,
          [-1.8674,  2.0761,  1.8723,  ..., -0.1823, -1.5117,  1.8220],
          [-2.6943,  2.3593,  2.3097,  ...,  0.0210, -1.7364,  1.0235],
          [-1.4583,  3.8678,  1.2712,  ..., -0.0709, -2.3355,  2.4812]],

         [[ 0.5789, -0.2984, -1.4066,  ...,  0.0523,  1.9950,  0.9737],
          [ 0.7331, -0.9596, -1.9101,  ...,  0.4358,  5.0752, -0.8699],
          [-1.2897, -3.2127, -3.0534,  ..., -0.2052,  1.8278,  2.7310],
          ...,
          [ 0.0382, -0.1829, -0.5204,  ...,  2.2170,  3.7642,  2.1352],
          [ 0.7929, -0.5755, -1.6306,  ..., -1.5692,  4.7004,  1.0015],
          [ 0.3076, -1.6089, -0.0630,  ..., -1.4793, 

In [21]:
# Saving the model
model.save_pretrained("./tmp/lora_gpt2")

In [20]:
#done.